# SCRAPPING GLOTELHO
## OBJECTIF
### récupérer le prix du xiaomi redmi note 11e pro chaque jour sur glotelho et me notifier quand
### il est <= 90000 FCFA

In [2]:
# Importons les bibliothèques nécéssaires
from bs4 import BeautifulSoup
import requests
import smtplib
import json
from email.message import EmailMessage
import time

Stockons l’URL du produit dans la variable url, puis récupèrons le contenu HTML de cette page via une requête HTTP GET, enfin analysons avec BeautifulSoup.

In [4]:
def scrape_data():
    url = 'https://glotelho.cm/product/xiaomi-redmi-note-11e-pro-5g-version-chinoise-6-67-128-go-8-go-ram-2-sim-108mp-8mp-2mp-16mp-5000mah-12-mois-de-garantie-46456'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')
    return soup

Après avoir analyser le code nous constatons que le prix et la quantité en stock sont dans des balises 'span',
avec les classes nous pouvons  donc récupérer ces information de la manière suivante :

In [6]:
def filter_data(soup):
    stock_span = soup.find_all('span', class_ = 'font-medium text-orange-400 md:text-[.85rem] md:border-2 xs:border-2 xs:text-center border-orange-400 md:rounded md:px-2 xs:rounded xs:px-1 xs:text-[0.8rem]' )[0]
    price_span = soup.find_all('span', class_ = 'xl:text-[27px] text-sm xs:text-[1.8rem] lg:text-[28px] lg:leading-normal md:text-[28px] font-bold text-gray-900 font-montserrat font-extrabold' )[0]
    return stock_span,price_span

Une fois les balises récupérés, nous extrayons et netoyons leurs contenus

In [8]:
def clean_data(stock_span,price_span):
    price = price_span.text.strip()
    in_stock = stock_span.text.strip()
    price = price.split(' ')[0]
    in_stock = in_stock.split('(')[1].split(')')[0]
    return price, in_stock

Transformons les données

In [10]:
def transform_data(price, in_stock):
    in_stock = int(in_stock) # entier
    price = float(price.replace(',','')) # nombre à virgule flottante
    return price, in_stock

Configurons l'envoie d'email 
Le fichier config.json doit être déja créer et être dans le même répertoire que notre fichier

In [12]:
def send_email(message="Hello! Le Xiaomi redmi pro est dispo à un bon prix !"):
    json_file = open("config.json")
    gmail_cfg = json.load(json_file)                 
    msg = EmailMessage()
    msg["to"] = gmail_cfg["email"]
    msg["from"] = gmail_cfg["email"]
    msg["Subject"] = "Notification Xiaomi redmi 11 pro"
    msg.set_content(message)
    with smtplib.SMTP_SSL(gmail_cfg["server"], gmail_cfg["port"]) as smtp:
        smtp.login(gmail_cfg["email"], gmail_cfg["pwd"])
        smtp.send_message(msg)
        print("Email envoyé !")


La methode principale qui orchestre toutes les autres pour mener à bien notre projet
Elle est configurer pour me notifier aussi en cas d'erreur

In [14]:
def main():
    try:
        soup = scrape_data()
        stock_span, price_span = filter_data(soup)
        price, in_stock = clean_data(stock_span, price_span)
        price, in_stock = transform_data(price, in_stock)

        print(f"Prix : {price} FCFA | En stock : {in_stock}")

        if price <= 90000 and in_stock > 0:
            send_email("Hello! Le Xiaomi redmi pro est dispo à un prix inférieur ou égal à 90 000 FCFA 🎉")

    except Exception as e:
        error_message = f"Erreur lors de l'exécution du script : {str(e)}"
        print(error_message)
        send_email(error_message)


Automatisons en appellant la methode principale une fois par jour

In [ ]:
while True:
    main()
    print("Attente de 24h...")
    time.sleep(86400) # 24h

Prix : 100000.0 FCFA | En stock : 0
Attente de 24h...
